# Mobility in Puerto Rico following Hurricane Maria

### Baseline model: Measuring travel time from all origins (~1 mil WorldPop cells) to key health facilities
Baseline travel time is measured as the walking time (accounting for slope) from origin Point A to the closest node 
on the road network, plus driving time from there to the closest road node to a service.
Facilities are expected to be proximal to the road network, so no measure is taken between 
road and service.

Health facilities datasets:
- dialysis facilities
- hospitals
- pharmacies

### Storm disruption model: Accounting for storm-related disruptions to baseline travel time.
The disruption model simulates travel slowdowns and alternate routing due to geohazards, infrastructural damage, and facility incapacity. Disrupted travel times are measured by i) full road obstruction requiring alternate routing and ii) road speed penalty affecting total travel time. 

Storm-related hazards indicating complete obstruction:
- flooding intersection at 3 feet depth
- concentration of more than 25 landslides per square kilometer

Storm-related hazard indicating road speed penalty:
- flooding intersection at 1 and 2 feet depth
- concentration of 1 - 25 landslides per square kilometer

### 1. Configure script.

In [1]:
import os, sys
GISFolder = os.getcwd()

In [2]:
# Note: gostnet.py and associated files are in the current working directory.

os.getcwd()

'C:\\Users\\grace\\GIS\\puerto rico'

In [3]:
import GOSTnet as gn # Python couldn't find the module. Moved it into C:\Users\grace\Anaconda3\envs\access\Lib\site-packages
import pandas as pd
from geopandas import GeoDataFrame
import shapely
from shapely.geometry import Point, box
import geopandas as gpd
import osmnx as ox
import networkx as nx
import numpy as np
import rasterio as rt

In [4]:
# Didn't use in this iteration:
import fiona
import peartree
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [5]:
pth = os.path.join(GISFolder, "data\working files\gn")
pth

'C:\\Users\\grace\\GIS\\puerto rico\\data\\working files\\gn'

### 2. Get driving network for all islands in Puerto Rico. 

Reloading OSM data from file. Origins and health facilities are not yet incorporated. Travel measured in length (meters).

In [6]:
gTime = nx.read_gpickle("gTime.pickle")

#### Spatial join in Python on the gdf 

In [7]:
gTime_node_gdf = gn.node_gdf_from_graph(gTime)
gTime_edge_gdf = gn.edge_gdf_from_graph(gTime)
# Takes just a few seconds.

C:\Users\grace\Anaconda3\envs\access\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
gTime_edge_gdf = gTime_edge_gdf.drop(['oneway','lanes','ref','maxspeed','area', 'width','service','junction','access','tunnel','name'], axis=1)

In [9]:
gTime_edge_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 393582 entries, 0 to 393581
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   stnode    393582 non-null  int64   
 1   endnode   393582 non-null  int64   
 2   osmid     393582 non-null  object  
 3   highway   393582 non-null  object  
 4   time      393582 non-null  float64 
 5   mode      393582 non-null  object  
 6   bridge    3165 non-null    object  
 7   length    393582 non-null  float64 
 8   geometry  393582 non-null  geometry
dtypes: float64(2), geometry(1), int64(2), object(4)
memory usage: 27.0+ MB


In [10]:
# This is the flood insurance zones dataset. Using the meters depth column (0,1,2, or 3 meters deep).
firz = os.path.join(pth, "FIRZ_meters.shp")
firz = gpd.read_file(firz)

In [13]:
firz.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2678 entries, 0 to 2677
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJECTID  2678 non-null   float64 
 1   FLD_ZONE  2678 non-null   object  
 2   DEP       2678 non-null   int64   
 3   geometry  2678 non-null   geometry
dtypes: float64(1), geometry(1), int64(1), object(1)
memory usage: 83.8+ KB


In [12]:
firz = firz[['OBJECTID', 'FLD_ZONE', 'DEP','geometry']]

In [14]:
firz.crs

<Projected CRS: PROJCS["NAD83_2011_Puerto_Rico_and_Virgin_Is",GEOG ...>
Name: NAD83_2011_Puerto_Rico_and_Virgin_Is
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [15]:
gTime_edge_gdf.crs

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
firz = firz.to_crs("EPSG:4326")

In [17]:
gTime_edge_gdf = gTime_edge_gdf.to_crs("EPSG:4326")

In [18]:
firz.crs == gTime_edge_gdf.crs

True

In [19]:
join = gpd.sjoin(gTime_edge_gdf, firz, how="left", op="intersects")
# This takes 30+ minutes.

In [20]:
join.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 422258 entries, 0 to 393581
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   stnode       422258 non-null  int64   
 1   endnode      422258 non-null  int64   
 2   osmid        422258 non-null  object  
 3   highway      422258 non-null  object  
 4   time         422258 non-null  float64 
 5   mode         422258 non-null  object  
 6   bridge       5240 non-null    object  
 7   length       422258 non-null  float64 
 8   geometry     422258 non-null  geometry
 9   index_right  422258 non-null  int64   
 10  OBJECTID     422258 non-null  float64 
 11  FLD_ZONE     422258 non-null  object  
 12  DEP          422258 non-null  int64   
dtypes: float64(3), geometry(1), int64(4), object(5)
memory usage: 45.1+ MB


In [28]:
# nx.to_networkx_graph?

In [21]:
gTime_firz_df = pd.DataFrame(join)

In [22]:
gTime_firz_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422258 entries, 0 to 393581
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   stnode       422258 non-null  int64   
 1   endnode      422258 non-null  int64   
 2   osmid        422258 non-null  object  
 3   highway      422258 non-null  object  
 4   time         422258 non-null  float64 
 5   mode         422258 non-null  object  
 6   bridge       5240 non-null    object  
 7   length       422258 non-null  float64 
 8   geometry     422258 non-null  geometry
 9   index_right  422258 non-null  int64   
 10  OBJECTID     422258 non-null  float64 
 11  FLD_ZONE     422258 non-null  object  
 12  DEP          422258 non-null  int64   
dtypes: float64(3), geometry(1), int64(4), object(5)
memory usage: 45.1+ MB


In [23]:
gTime_firz_df.to_csv(os.path.join(pth, 'gTime_FIRZ.csv'))

#### Create speed penalties.
This is easier in R, but I want to avoid saving and reloading the file due to the pickiness of converting back to a graph.

In [25]:
gTime_firz_df["penalty_f"] = 0

In [33]:
gTime_firz_df.head(10) # Time is in seconds.

,index,stnode,endnode,osmid,highway,time,mode,bridge,length,geometry,index_right,OBJECTID,FLD_ZONE,DEP,penalty_f
0,0,238026752,238075156,22137085,residential,19.56906,drive,NaN,108.717,"LINESTRING (-66.05025 18.26190, -66.04995 18.2...",1123,4131840.0,X,1,24.461325
1,1,238026752,238026733,22128613,residential,34.03782,drive,NaN,189.099,"LINESTRING (-66.05025 18.26190, -66.05042 18.2...",1123,4131840.0,X,1,42.547275
2,2,238026752,238072036,"[22136924, 22137599]",residential,30.33810,drive,NaN,168.545,"LINESTRING (-66.05025 18.26190, -66.04985 18.2...",1123,4131840.0,X,1,37.922625
3,3,238026752,237995790,22137599,residential,12.39138,drive,NaN,68.841,"LINESTRING (-66.05025 18.26190, -66.05060 18.2...",1123,4131840.0,X,1,15.489225
4,4,238026754,4220614634,422318208,tertiary,2.56236,drive,NaN,21.353,"LINESTRING (-65.99028 18.42781, -65.99011 18.4...",2658,4132302.0,A,3,2.562360
5,5,238026754,238026757,196946625,tertiary,4.86816,drive,NaN,40.568,"LINESTRING (-65.99028 18.42781, -65.99034 18.4...",2658,4132302.0,A,3,4.868160
6,5,238026754,238026757,196946625,tertiary,4.86816,drive,NaN,40.568,"LINESTRING (-65.99028 18.42781, -65.99034 18.4...",2299,4132149.0,X,1,6.085200
7,6,238026754,4220614614,"[422318209, 196950642]",unclassified,7.94970,drive,yes,44.165,"LINESTRING (-65.99028 18.42781, -65.99022 18.4...",2658,4132302.0,A,3,7.949700
8,7,238026757,238050840,22132255,tertiary,84.36504,drive,NaN,703.042,"LINESTRING (-65.99061 18.42800, -65.99077 18.4...",2299,4132149.0,X,1,105.456300
9,8,238026757,238026754,196946625,tertiary,4.86816,drive,NaN,40.568,"LINESTRING (-65.99061 18.42800, -65.99054 18.4...",2658,4132302.0,A,3,4.868160


In [27]:
gTime_firz_df.reset_index(inplace=True)

In [29]:
gTime_firz_df["penalty_f"] = gTime_firz_df['time']
# ValueError: cannot reindex from a duplicate axis. Needed to do reset_index (see previous line).

In [31]:
gTime_firz_df.loc[(gTime_firz_df['DEP'] == 1), # Select rows where depth is 1 foot, select just the penalty column.
       'penalty_f'] = (gTime_firz_df['time'] * 1.25) # then slow down slightly

In [34]:
gTime_firz_df.loc[(gTime_firz_df['DEP'] == 2),
       'penalty_f'] = (gTime_firz_df['time'] * 2) # then double time

In [35]:
gTime_firz_df.loc[(gTime_firz_df['DEP'] == 3),
       'penalty_f'] = (gTime_firz_df['time'] * 5) # then increase time 5 fold

In [36]:
gTime_firz_df.tail()

,index,stnode,endnode,osmid,highway,time,mode,bridge,length,geometry,index_right,OBJECTID,FLD_ZONE,DEP,penalty_f
422253,393579,238026748,5167073092,196950650,tertiary,39.56712,drive,NaN,329.726,"LINESTRING (-65.98955 18.42824, -65.98967 18.4...",2658,4132302.0,A,3,197.835600
422254,393579,238026748,5167073092,196950650,tertiary,39.56712,drive,NaN,329.726,"LINESTRING (-65.98955 18.42824, -65.98967 18.4...",2299,4132149.0,X,1,49.458900
422255,393580,238026748,4220614629,"[22136906, 363774843, 422318206]",tertiary,5.76624,drive,yes,48.052,"LINESTRING (-65.98955 18.42824, -65.98948 18.4...",2658,4132302.0,A,3,28.831200
422256,393581,238026748,238026729,532490641,unclassified,37.04274,drive,NaN,205.793,"LINESTRING (-65.98955 18.42824, -65.98945 18.4...",2658,4132302.0,A,3,185.213700
422257,393581,238026748,238026729,532490641,unclassified,37.04274,drive,NaN,205.793,"LINESTRING (-65.98955 18.42824, -65.98945 18.4...",2299,4132149.0,X,1,46.303425


#### Convert back to graph

In [37]:
gTime2 = nx.from_pandas_edgelist(
    gTime_firz_df,
    source="stnode",
    target="endnode",
    edge_attr=True,
)
# It ran. Only took a few seconds.

In [38]:
gn.example_edge(gTime2, 10)

(238026752, 238075156, {'index': 39981, 'osmid': 22137085, 'highway': 'residential', 'time': 19.569059999999997, 'mode': 'drive', 'bridge': nan, 'length': 108.717, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001FE2C7CBAC8>, 'index_right': 1123, 'OBJECTID': 4131840.0, 'FLD_ZONE': 'X', 'DEP': 1, 'penalty_f': 24.461324999999995})
(238026752, 238026733, {'index': 393574, 'osmid': 22128613, 'highway': 'residential', 'time': 34.03782, 'mode': 'drive', 'bridge': nan, 'length': 189.099, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001FE58E8E160>, 'index_right': 1123, 'OBJECTID': 4131840.0, 'FLD_ZONE': 'X', 'DEP': 1, 'penalty_f': 42.547275000000006})
(238026752, 238072036, {'index': 37001, 'osmid': [22136924, 22137599], 'highway': 'residential', 'time': 30.338100000000004, 'mode': 'drive', 'bridge': nan, 'length': 168.54500000000002, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001FE33AEC4A8>, 'index_right': 1123, 'OBJECTID': 41

In [42]:
gn.save(gTime2, 'gTime_firz', pth, edges = False, nodes = False)

In [41]:
gn.save?

### 3. Origins and destinations

Measure distance from origin/destination to nearest node and save to file.

In [43]:
#%% If already created, load from file.
inOsnap = os.path.join(GISFolder, "data\working files\gn", "inOsnap.csv")
inOsnap = pd.read_csv(inOsnap)
inDsnap = os.path.join(GISFolder, "data\working files\gn", "inDsnap.csv")
inDsnap = pd.read_csv(inDsnap)
inHsnap = os.path.join(GISFolder, "data\working files\gn", "inHsnap.csv")
inHsnap = pd.read_csv(inHsnap)
inPsnap = os.path.join(GISFolder, "data\working files\gn", "inPsnap.csv")
inPsnap = pd.read_csv(inPsnap)

### Create flood-affected travel time values for the road nodes nearest to each service.

Using calculate_OD.

In [44]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
origins = list(inOsnap.NN.unique())

In [45]:
listD = list(inDsnap.NN.unique()) 
listH = list(inHsnap.NN.unique()) 
listP = list(inPsnap.NN.unique()) 
destslist = listD + listH + listP
dests = list(set(destslist))

In [46]:
len(origins) # 142,421 unique nearest nodes.

142421

In [47]:
len(dests) # 1,011 unique nearest nodes.

1011

In [48]:
len(listH)

69

In [49]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [50]:
gn.example_edge(gTime2)

(238026752, 238075156, {'index': 39981, 'osmid': 22137085, 'highway': 'residential', 'time': 19.569059999999997, 'mode': 'drive', 'bridge': nan, 'length': 108.717, 'geometry': <shapely.geometry.linestring.LineString object at 0x000001FE2C7CBAC8>, 'index_right': 1123, 'OBJECTID': 4131840.0, 'FLD_ZONE': 'X', 'DEP': 1, 'penalty_f': 24.461324999999995})


In [51]:
importlib.reload(gn)

<module 'GOSTnet' from 'C:\\Users\\grace\\Anaconda3\\envs\\access\\lib\\site-packages\\GOSTnet.py'>

In [52]:
OD_flood = gn.calculate_OD(gTime2, origins, dests, fail_value, weight = 'penalty_f')
# Took maybe 10 min.

In [53]:
OD_flood_df = pd.DataFrame(OD_flood, index = origins, columns = dests)

In [54]:
OD_flood_df.info()
# Created a 142421 x 1011 matrix. Same dimensions and size as the non-flood (1.1 GB)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142421 entries, 239148931 to 238880909
Columns: 1011 entries, 233324546 to 238196735
dtypes: float64(1011)
memory usage: 1.1 GB


In [56]:
OD_flood_df.tail()

,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,238549012,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
238421388,28834.718732,13877.572348,25009.894285,8562.560637,6085.473590,13884.491173,8096.600155,8446.633325,15297.398766,14333.150960,...,5747.060487,25036.259995,8279.930468,6418.39862,9179.057290,8644.031630,8612.450988,13915.057198,9443.002105,11534.368457
238402093,28846.119632,13888.973248,25021.295185,8573.961537,6096.874490,13895.892073,8108.001055,8458.034225,15308.799666,14344.551860,...,5758.461388,25047.660895,8291.331368,6429.79952,9190.458190,8655.432530,8623.851887,13926.458098,9454.403005,11545.769357
238591938,26119.772406,13883.800435,21488.637576,8568.788725,1660.176637,13890.719260,8102.828243,8452.861412,11776.142057,14339.379048,...,5753.288575,21515.003286,8286.158555,6430.73417,9185.285378,8650.259718,8618.679075,13921.285285,9449.230193,11540.596545
238923898,20345.327300,13309.129070,15714.192470,8023.938073,8718.020753,13316.047895,7530.686328,7878.190048,6001.696951,13764.707683,...,6491.293260,15740.558180,7740.647235,5842.74275,8639.774057,8104.748398,9755.685508,13346.613920,8874.558828,10475.748495
238880909,20600.886350,13564.688120,15969.751520,8279.497122,8973.579803,13571.606945,7786.245378,8133.749097,6257.256001,14020.266733,...,6746.852310,15996.117230,7996.206285,6098.30180,8895.333107,8360.307447,10011.244558,13602.172970,9130.117878,10731.307545


In [57]:
# Convert to minutes and save to file.
OD_flood_min = OD_flood_df[OD_flood_df <fail_value] / 60
OD_flood_min.to_csv(os.path.join(pth, 'OD_flood.csv'))
# Takes a couple minutes.

In [58]:
OD_flood_min.tail(20)

,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,238549012,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
238061534,358.427270,109.375592,313.641292,60.749743,169.810707,109.490906,51.528422,43.765175,227.834259,116.968569,...,90.503160,313.358605,49.088817,78.639203,52.546505,49.927639,82.990721,110.000340,52.583213,76.415749
6242335058,355.252829,106.165597,310.466851,19.037110,142.431259,106.280911,28.395784,29.146603,200.940812,113.758574,...,63.123713,310.184164,15.932077,51.320869,2.641398,14.322802,34.971810,106.790345,44.491211,73.205753
238061543,356.307226,107.255548,311.521248,57.983742,167.044706,107.370862,48.762421,40.999174,225.068258,114.848525,...,87.737160,311.238561,46.322816,75.873203,49.780504,47.161638,80.224721,107.880295,50.463168,74.295704
5148259911,353.321723,104.270045,308.535745,49.604805,158.665769,104.385359,40.383484,32.620237,216.689321,111.863022,...,79.358223,308.253058,37.943879,67.494266,41.401567,38.782701,71.845784,104.894793,47.477666,71.310201
238061666,348.141831,99.090153,303.355853,44.706733,153.767697,99.205467,35.485412,27.722165,211.791249,106.683130,...,74.460151,303.073166,33.045807,62.596194,36.503495,33.884629,66.947712,99.714901,42.297774,66.130309
238061667,347.986156,98.934478,303.200178,44.269238,153.330202,99.049792,35.047917,27.284670,211.353754,106.527455,...,74.022656,302.917491,32.608312,62.158699,36.066000,33.447134,66.510217,99.559226,42.142099,65.974634
238065788,354.914743,105.863065,310.128765,50.422638,159.483602,105.978379,41.201317,33.438070,217.507154,113.456042,...,80.176056,309.846078,38.761712,68.312099,42.219400,39.600534,72.663617,106.487813,48.519443,72.903221
238349709,350.222104,101.170426,305.436126,45.729999,154.790963,101.285740,36.508678,28.745431,212.814515,108.763403,...,75.483417,305.153439,34.069073,63.619460,37.526761,34.907895,67.970978,101.795174,43.826804,68.210582
238347326,347.511004,98.459326,302.725026,43.018899,152.079863,98.574640,33.797578,26.034331,210.103415,106.052303,...,72.772317,302.442339,31.357973,60.908360,34.815661,32.196795,65.259878,99.084074,41.115704,65.499482
238261379,345.756229,96.704551,300.970251,41.264124,150.325088,96.819865,32.042803,24.279556,208.348640,104.297528,...,71.017542,300.687564,29.603198,59.153585,33.060886,30.442020,63.505103,97.329299,39.360929,63.744707


In [59]:
# Create POI-specific OD and save to file.
ODD_flood = OD_flood_df.loc[:, listD]
ODD_flood = ODD_flood[ODD_flood < fail_value] / 60 
ODD_flood.to_csv(os.path.join(pth, 'ODD_flood.csv')) # Each takes a minute or so.

ODH_flood = OD_flood_df.loc[:, listH]
ODH_flood = ODH_flood[ODH_flood < fail_value] / 60 
ODH_flood.to_csv(os.path.join(pth, 'ODH_flood.csv'))

ODP_flood = OD_flood_df.loc[:, listP]
ODP_flood = ODP_flood[ODP_flood < fail_value] / 60 
ODP_flood.to_csv(os.path.join(pth, 'ODP_flood.csv'))

In [60]:
ODP_flood.tail()

,238905285,238981133,2984408674,238960930,238067397,238363165,237993610,238174882,238556281,238468195,...,239020197,4198083801,243624306,237973577,237838093,237909639,499860461,239051712,238934844,238979708
238421388,131.427702,140.777222,145.683140,147.378309,154.381795,97.775683,112.077314,207.351949,383.168171,378.938695,...,204.378381,282.176469,114.971354,157.901681,293.262979,341.653553,149.163482,164.615873,161.350570,149.085758
238402093,131.617717,140.967237,145.873155,147.568324,154.571810,97.965698,112.267329,207.541964,383.358186,379.128710,...,204.568396,282.366484,115.161369,158.091696,293.452994,341.843568,149.353497,164.805888,161.540585,149.275773
238591938,131.531504,140.881024,145.786941,147.482110,154.485596,97.879484,112.217844,207.455751,324.480559,320.251083,...,145.148120,223.488858,73.328571,158.005483,293.366780,341.757355,149.267283,164.719674,161.454372,149.189560
238923898,122.108030,131.303167,136.209085,138.018622,145.393741,111.759829,94.769826,158.248962,228.239807,224.010331,...,48.660673,127.248106,124.928121,148.427627,231.474417,270.091457,168.407690,155.141818,151.990883,140.097704
238880909,126.367347,135.562485,140.468403,142.277939,149.653058,116.019147,99.029144,162.508279,232.499125,228.269649,...,52.919990,131.507423,129.187438,152.686944,235.733734,274.350774,172.667008,159.401136,156.250200,144.357022


### Compare flooding and baseline results.

In [61]:
# Run this even if already loaded. Name change for flood file.
OD_base = os.path.join(pth, "OD.csv")
OD_base = pd.read_csv(OD_base)

In [65]:
OD_flood = os.path.join(pth, "OD_flood.csv")
OD_flood = pd.read_csv(OD_flood)

In [62]:
OD_base

,Unnamed: 0,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
0,239148931,114.824141,110.959791,65.620551,184.517029,218.033975,112.014890,177.794677,171.425579,85.948924,...,204.024088,66.515588,175.284588,195.431551,178.777999,177.272204,202.057806,110.459993,170.807127,147.563904
1,239077120,134.197077,107.545973,84.993487,181.103212,225.610682,108.601072,174.380859,168.011761,93.525632,...,205.775851,85.888524,171.870771,196.361803,175.364182,173.858386,198.643989,107.046175,167.393310,144.150087
2,239127767,131.989053,105.337949,82.785463,178.895188,223.402658,106.393048,172.172835,165.803737,91.317608,...,203.567827,83.680500,169.662747,194.153779,173.156158,171.650362,196.435965,104.838151,165.185286,141.942063
3,239136364,131.674608,105.023504,82.471018,178.580743,223.088213,106.078603,171.858390,165.489292,91.003163,...,203.253382,83.366055,169.348302,193.839334,172.841713,171.335917,196.121520,104.523706,164.870841,141.627618
4,239077188,132.201900,105.550796,82.998310,179.108035,223.615505,106.605895,172.385682,166.016584,91.530455,...,203.780674,83.893347,169.875594,194.366626,173.369005,171.863209,196.648812,105.050998,165.398133,142.154910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142416,238421388,274.348489,150.726605,262.410984,86.810998,39.038662,150.839943,80.654407,84.603840,172.531964,...,48.695262,261.493238,83.006299,57.307928,96.245695,87.874784,80.742644,151.216019,98.125120,122.596289
142417,238402093,274.386492,150.764608,262.448987,86.849001,39.076665,150.877946,80.692410,84.641843,172.569967,...,48.733265,261.531241,83.044302,57.345931,96.283698,87.912787,80.780647,151.254022,98.163123,122.634292
142418,238591938,263.079305,164.817016,247.437265,100.901409,22.135689,164.930354,94.744819,98.694252,127.237173,...,62.785673,248.332302,97.096710,71.398340,110.336106,101.965196,94.833056,165.306430,112.215532,136.686700
142419,238923898,209.664729,169.891442,194.022690,105.975835,91.021763,170.004780,99.819245,103.768677,73.822598,...,85.535147,194.917727,102.171136,76.942609,115.410532,107.039621,128.237466,170.380856,117.289957,138.778429


In [66]:
OD_flood

,Unnamed: 0,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
0,239148931,159.879130,201.743395,82.693549,297.054865,319.683265,203.062269,288.834002,286.236251,119.643593,...,271.510785,83.132978,291.214029,260.701610,295.017581,292.052851,325.115933,201.118648,287.541754,248.508994
1,239077120,197.077463,186.016039,119.891882,292.895063,328.508907,187.334913,283.121290,276.256359,128.469236,...,280.336427,120.331311,281.234137,269.527252,285.002136,282.072960,315.136042,185.391292,277.327767,235.636509
2,239127767,194.317433,183.256009,117.131852,290.135033,325.748877,184.574883,280.361260,273.496329,125.709206,...,277.576397,117.571281,278.474107,266.767222,282.242106,279.312930,312.376012,182.631262,274.567737,232.876479
3,239136364,193.924376,182.862953,116.738796,289.741977,325.355821,184.181827,279.968204,273.103273,125.316149,...,277.183341,117.178224,278.081051,266.374166,281.849050,278.919873,311.982956,182.238206,274.174681,232.483422
4,239077188,194.583492,183.522068,117.397911,290.401092,326.014936,184.840942,280.627319,273.762388,125.975264,...,277.842456,117.837339,278.740166,267.033281,282.508165,279.578988,312.642071,182.897321,274.833796,233.142537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142416,238421388,480.578646,231.292872,416.831571,142.709344,101.424560,231.408186,134.943336,140.777222,254.956646,...,95.784341,417.271000,137.998841,106.973310,152.984288,144.067194,143.540850,231.917620,157.383368,192.239474
142417,238402093,480.768661,231.482887,417.021586,142.899359,101.614575,231.598201,135.133351,140.967237,255.146661,...,95.974356,417.461015,138.188856,107.163325,153.174303,144.257209,143.730865,232.107635,157.573383,192.429489
142418,238591938,435.329540,231.396674,358.143960,142.813145,27.669611,231.511988,135.047137,140.881024,196.269034,...,95.888143,358.583388,138.102643,107.178903,153.088090,144.170995,143.644651,232.021421,157.487170,192.343276
142419,238923898,339.088788,221.818818,261.903208,133.732301,145.300346,221.934132,125.511439,131.303167,100.028283,...,108.188221,262.342636,129.010787,97.379046,143.996234,135.079140,162.594758,222.443565,147.909314,174.595808


In [67]:
OD_base == OD_flood
# Returns True/False matrix

,Unnamed: 0,233324546,4155721731,238944260,239097861,239310852,4155721735,238962698,238981133,238845965,...,238372836,238995433,763959274,238055405,238071790,2149629933,238018545,4155721720,237981689,238196735
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142416,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
142417,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
142418,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
142419,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [68]:
((OD_base == OD_flood).sum()).sum()
# Number of matching values. 

145424

In [69]:
((OD_base == OD_flood).sum()).sum() / ((OD_base == OD_base).sum()).sum()
# Percent of routes remaining the same after the flood simulation.

0.001010570982194746

### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [71]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
ODD_flood = os.path.join(pth, "ODD_flood.csv")
ODD_flood = pd.read_csv(ODD_flood)
ODP_flood = os.path.join(pth, "ODP_flood.csv")
ODP_flood = pd.read_csv(ODP_flood)
ODH_flood = os.path.join(pth, "ODH_flood.csv")
ODH_flood = pd.read_csv(ODH_flood)

In [80]:
ODD_flood.tail(10)

,NN,243545119,238600357,237883590,237875741,237992953,4377866427,238060667,553993016,239097861,...,238957886,2903017551,238975872,238997632,238965771,238065313,4155927397,239209720,5136150615,239344045
142411,238547482,387.029218,397.045605,293.092267,309.565748,103.359545,114.987573,112.558937,37.364216,71.754307,...,80.445459,81.950735,84.170100,77.738066,380.745955,103.746727,144.222218,160.814552,88.345694,370.538092
142412,238537241,394.938284,404.954671,301.001333,317.474814,111.268610,122.896638,120.468003,45.273282,79.663373,...,88.354524,89.859801,92.079165,85.647132,388.655021,111.655793,152.131283,168.723618,68.762827,378.447158
142413,5138198157,394.540285,404.556672,300.603334,317.076815,110.870612,122.498640,120.070004,44.875283,79.265374,...,87.956525,89.461802,91.681166,85.249133,388.257022,111.257794,151.733284,168.325619,68.364828,378.049159
142414,238521463,393.361225,403.377612,299.424274,315.897755,109.691552,121.319580,118.890944,43.696223,78.086314,...,86.777465,88.282742,90.502106,84.070073,387.077962,110.078734,150.554224,167.146559,67.185768,376.870099
142415,238217078,422.308982,432.325369,328.372031,344.845512,138.639308,125.550208,128.396900,72.643980,107.034071,...,115.725222,117.230499,119.449863,113.017830,416.025719,139.026491,179.501981,196.094316,68.430773,386.343147
142416,238421388,444.542763,454.559150,350.605812,367.079293,153.417862,105.985764,108.832455,146.649490,142.709344,...,134.271621,137.416115,139.577056,145.288041,420.020519,161.385092,197.897587,218.094002,128.896558,366.778702
142417,238402093,444.732778,454.749165,350.795827,367.269308,153.607877,106.175779,109.022470,146.839505,142.899359,...,134.461636,137.606130,139.767071,145.478056,420.210534,161.575107,198.087602,218.284017,129.086573,366.968717
142418,238591938,431.367260,403.443162,350.709614,330.843216,153.521664,106.126293,108.972985,146.753291,142.813145,...,134.375422,137.519916,139.680857,145.391842,361.332907,161.488894,198.001388,218.197804,134.477690,308.091090
142419,238923898,335.126508,307.202410,257.144448,234.602464,143.943808,93.673184,91.524968,165.893698,133.732301,...,125.294578,127.976560,130.103001,136.299987,265.092156,151.911038,188.423532,208.619947,220.801129,211.850339
142420,238880909,339.385825,311.461727,261.403765,238.861782,148.203125,97.932502,95.784285,170.153016,137.991619,...,129.553896,132.235878,134.362319,140.559304,269.351473,156.170355,192.682850,212.879265,225.060447,216.109656


In [79]:
ODD_flood.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) # Repeat for each OD set, if needed.

#### Find first nearest POI for each origin node. Run this block for each variable.

In [81]:
ODD_flood["1D"] = 0
sub = ODD_flood.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
ODD_flood["1D"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
D1_flood = ODD_flood[['NN', '1D']] # Remove unnecessary OD values.
D1_flood.to_csv(os.path.join(pth, '1D_flood.csv'))

In [82]:
D1_flood.head(10) # Validation: The nodes that had NaN values in the OD matrix show a higher travel time in the landslide set. 

,NN,1D
0,239148931,47.234415
1,239077120,82.027676
2,239127767,79.267646
3,239136364,78.874590
4,239077188,79.533705
5,239176627,46.048155
6,237676072,50.124027
7,239149053,76.450020
8,237686017,84.884104
9,239158919,42.438336


In [83]:
ODH_flood["1H"] = 0
sub = ODH_flood.iloc[:,1:-1] # Filtering out the newly created field and the node ID column.
ODH_flood["1H"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
H1_flood = ODH_flood[['NN', '1H']] # Remove unnecessary OD values.
H1_flood.to_csv(os.path.join(pth, '1H_flood.csv'))

In [84]:
H1_flood.head(10)

,NN,1H
0,239148931,49.039920
1,239077120,52.009464
2,239127767,49.249434
3,239136364,48.856378
4,239077188,49.515493
5,239176627,47.853660
6,237676072,35.492887
7,239149053,46.431808
8,237686017,54.865892
9,239158919,35.502936


In [85]:
ODP_flood["1P"] = 0
sub = ODP_flood.iloc[:,1:-1] # Filtering out the newly created field and the node ID column.
ODP_flood["1P"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
P1_flood = ODP_flood[['NN', '1P']] # Remove unnecessary OD values.
P1_flood.to_csv(os.path.join(pth, '1P_flood.csv'))

In [86]:
P1_flood.head(10)

,NN,1P
0,239148931,17.828370
1,239077120,24.581559
2,239127767,21.821529
3,239136364,21.428473
4,239077188,22.087588
5,239176627,16.642110
6,237676072,35.492887
7,239149053,19.003902
8,237686017,27.437986
9,239158919,31.054246


#### Compare to baseline values.

In [95]:
D1_base = os.path.join(pth, "1D.csv")
D1_base = pd.read_csv(D1_base)
P1_base = os.path.join(pth, "1P.csv")
P1_base = pd.read_csv(P1_base)
H1_base = os.path.join(pth, "1H.csv")
H1_base = pd.read_csv(H1_base)

In [100]:
P1_base.head()

,NN,1P
0,239148931,14.262696
1,239077120,19.665247
2,239127767,17.457223
3,239136364,17.142778
4,239077188,17.670070


In [97]:
P1_base = P1_base.loc[:,['NN', '1P']] # Remove unnecessary Unnamed: 0 column.
D1_base = D1_base.loc[:,['NN', '1D']]
H1_base = H1_base.loc[:,['NN', '1H']]

In [101]:
P1_flood.head()

,NN,1P
0,239148931,17.828370
1,239077120,24.581559
2,239127767,21.821529
3,239136364,21.428473
4,239077188,22.087588


In [102]:
P1_base == P1_flood

,NN,1P
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
142416,True,False
142417,True,False
142418,True,False
142419,True,False


In [103]:
((P1_base['1P'] == P1_flood['1P']).sum()).sum()
# Number of routes that remained the same.

1477

In [105]:
((P1_base['1P'] == P1_flood['1P']).sum()).sum() / ((P1_base['1P'] == P1_base['1P']).sum()).sum()
# 1.03 percent of routes remained the same after the flood simulation.

0.01037590710156025

In [106]:
((D1_base['1D'] == D1_flood['1D']).sum()).sum()

168

In [107]:
((D1_base['1D'] == D1_flood['1D']).sum()).sum() / ((D1_base['1D'] == D1_base['1D']).sum()).sum()
# 1.18 percent of routes remained the same after the flood simulation.

0.0011802891708468574

In [108]:
((H1_base['1H'] == H1_flood['1H']).sum()).sum()

79

In [109]:
((H1_base['1H'] == H1_flood['1H']).sum()).sum() / ((H1_base['1H'] == H1_base['1H']).sum()).sum()
# 0.05 percent of routes remained the same after the flood simulation.

0.000555016931529177

### Create multi-modal travel times by combining walk time to road with drive time to nth nearest service.

In [110]:
# If starting new session, re-load from disk.
zwalk = os.path.join(pth, "zwalk.csv") 
zwalk = pd.read_csv(zwalk)

In [59]:
D1_flood = os.path.join(pth, "1D_flood.csv")
D1_flood = pd.read_csv(D1_flood)

In [111]:
zwalk.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183


In [112]:
D1_flood.head()

,NN,1D
0,239148931,47.234415
1,239077120,82.027676
2,239127767,79.267646
3,239136364,78.874590
4,239077188,79.533705


In [113]:
# Merge nearest POIs and walktimes
zwalkD_flood = zwalk.merge(D1_flood, how='left', left_on='NN', right_on='NN', sort=False)
zwalkD_flood.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1D
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,47.234415
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,47.234415
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,47.234415
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,47.234415
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,82.027676


In [114]:
# Merge nearest POIs and walktimes
zwalkP_flood = zwalk.merge(P1_flood, how='left', left_on='NN', right_on='NN', sort=False)
zwalkP_flood.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1P
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,17.828370
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,17.828370
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,17.828370
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,17.828370
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,24.581559


In [115]:
# Merge nearest POIs and walktimes
zwalkH_flood = zwalk.merge(H1_flood, how='left', left_on='NN', right_on='NN', sort=False)
zwalkH_flood.head()

,Unnamed: 0,wpop,xmid,wid,municipio,NN,osmid,walkspeed,walk_time,1H
0,0,0.818646,0.0,1,Adjuntas,239148931,239148931,2.284031,9.998605,49.039920
1,1,0.731308,0.0,2,Adjuntas,239148931,239148931,3.250490,8.260954,49.039920
2,2,0.642141,0.0,3,Adjuntas,239148931,239148931,4.286563,7.290488,49.039920
3,3,0.612746,0.0,4,Adjuntas,239148931,239148931,4.613557,7.779941,49.039920
4,4,0.699177,0.0,5,Adjuntas,239077120,239077120,3.360424,12.971183,52.009464


In [ ]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkH_flood["mm1H"] = 0
zwalkH_flood["mm1H"] = zwalkH_flood["walk_time"] + zwalkH_flood["1H"]

In [120]:
zwalkH_flood.head()

,wpop,xmid,wid,municipio,NN,walk_time,1H,mm1H
0,0.818646,0.0,1,Adjuntas,239148931,9.998605,49.039920,59.038525
1,0.731308,0.0,2,Adjuntas,239148931,8.260954,49.039920,57.300874
2,0.642141,0.0,3,Adjuntas,239148931,7.290488,49.039920,56.330408
3,0.612746,0.0,4,Adjuntas,239148931,7.779941,49.039920,56.819861
4,0.699177,0.0,5,Adjuntas,239077120,12.971183,52.009464,64.980648


In [ ]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkP_flood["mm1P"] = 0
zwalkP_flood["mm1P"] = zwalkP_flood["walk_time"] + zwalkP_flood["1P"]

In [121]:
zwalkP_flood.head()

,wpop,xmid,wid,municipio,NN,walk_time,1P,mm1P
0,0.818646,0.0,1,Adjuntas,239148931,9.998605,17.828370,27.826975
1,0.731308,0.0,2,Adjuntas,239148931,8.260954,17.828370,26.089324
2,0.642141,0.0,3,Adjuntas,239148931,7.290488,17.828370,25.118858
3,0.612746,0.0,4,Adjuntas,239148931,7.779941,17.828370,25.608311
4,0.699177,0.0,5,Adjuntas,239077120,12.971183,24.581559,37.552742


In [ ]:
# Combine walk time from WorldPop point to nearest road node, and from road node to facility.
zwalkD_flood["mm1D"] = 0
zwalkD_flood["mm1D"] = zwalkD_flood["walk_time"] + zwalkD_flood["1D"]

In [122]:
zwalkD_flood.head()

,wpop,xmid,wid,municipio,NN,walk_time,1D,mm1D
0,0.818646,0.0,1,Adjuntas,239148931,9.998605,47.234415,57.233020
1,0.731308,0.0,2,Adjuntas,239148931,8.260954,47.234415,55.495369
2,0.642141,0.0,3,Adjuntas,239148931,7.290488,47.234415,54.524903
3,0.612746,0.0,4,Adjuntas,239148931,7.779941,47.234415,55.014356
4,0.699177,0.0,5,Adjuntas,239077120,12.971183,82.027676,94.998860


In [119]:
zwalkD_flood = zwalkD_flood[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1D', 'mm1D']]
zwalkH_flood = zwalkH_flood[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1H', 'mm1H']]
zwalkP_flood = zwalkP_flood[['wpop', 'xmid', 'wid', 'municipio', 'NN', 'walk_time', '1P', 'mm1P']]

In [123]:
zwalkH_flood.to_csv(os.path.join(pth, 'H_mm_flood.csv'))
zwalkP_flood.to_csv(os.path.join(pth, 'P_mm_flood.csv'))
zwalkD_flood.to_csv(os.path.join(pth, 'D_mm_flood.csv'))